In [1]:
import os
import sys
import urllib.request

def get_movie(movie_name_list):
    client_id = "viO54DlhW87ESywUDwWb"
    client_secret = "7ZHAscDgKs"
    for i in movie_name_list:
        encText = urllib.parse.quote('"겨울왕국"')
        # url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
        url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            print(response_body.decode('utf-8'))
        else:
            print("Error Code:" + rescode)

{
	"lastBuildDate":"Mon, 20 Mar 2023 20:17:52 +0900",
	"total":6,
	"start":1,
	"display":6,
	"items":[
		{
			"title":"<b>겨울왕국<\/b> 2",
			"link":"https:\/\/movie.naver.com\/movie\/bi\/mi\/basic.nhn?code=136873",
			"image":"https:\/\/ssl.pstatic.net\/imgmovie\/mdi\/mit110\/1368\/136873_P18_100537.jpg",
			"subtitle":"Frozen 2",
			"pubDate":"2019",
			"director":"크리스 벅|제니퍼 리|",
			"actor":"크리스틴 벨|이디나 멘젤|",
			"userRating":"8.69"
		},
		{
			"title":"<b>겨울왕국<\/b>의 무민",
			"link":"https:\/\/movie.naver.com\/movie\/bi\/mi\/basic.nhn?code=163850",
			"image":"https:\/\/ssl.pstatic.net\/imgmovie\/mdi\/mit110\/1638\/163850_P01_101514.jpg",
			"subtitle":"Moomins and the Winter Wonderland",
			"pubDate":"2017",
			"director":"이라 카르페란|야쿱 론스키|",
			"actor":"빌 스카스가드|알리시아 비칸데르|스텔란 스카스가드|",
			"userRating":"7.42"
		},
		{
			"title":"올라프의 <b>겨울왕국<\/b> 어드벤처",
			"link":"https:\/\/movie.naver.com\/movie\/bi\/mi\/basic.nhn?code=164140",
			"image":"https:\/\/ssl.pstatic.net\/imgmovie\/mdi\/mit110\/1

In [2]:
encText = urllib.parse.quote('"겨울왕국"')
# url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # JSON 결과
url = "https://openapi.naver.com/v1/search/movie.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"><channel><title>Naver Open API - movie ::&apos;&amp;quot;겨울왕국&amp;quot;&apos;</title><link>https://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Mon, 20 Mar 2023 20:22:49 +0900</lastBuildDate><total>6</total><start>1</start><display>6</display><item><title>&lt;b&gt;겨울왕국&lt;/b&gt; 2</title><link>https://movie.naver.com/movie/bi/mi/basic.nhn?code=136873</link><image>https://ssl.pstatic.net/imgmovie/mdi/mit110/1368/136873_P18_100537.jpg</image><subtitle>Frozen 2</subtitle><pubDate>2019</pubDate><director>크리스 벅|제니퍼 리|</director><actor>크리스틴 벨|이디나 멘젤|</actor><userRating>8.69</userRating></item><item><title>&lt;b&gt;겨울왕국&lt;/b&gt;의 무민</title><link>https://movie.naver.com/movie/bi/mi/basic.nhn?code=163850</link><image>https://ssl.pstatic.net/imgmovie/mdi/mit110/1638/163850_P01_101514.jpg</image><subtitle>Moomins and the Winter Wonderland</subtitle><pubDate>2017</pubDate><director>이라 카르페란|야쿱 론스키|</dir

In [56]:
response = urllib.request.urlopen(request)
response_body = response.read()
xml_data = response_body.decode('utf-8')

In [57]:
import xmltodict
dict_data = xmltodict.parse(xml_data, xml_attribs=False)

In [58]:
dict_data

{'rss': {'channel': {'title': "Naver Open API - movie ::'&quot;겨울왕국&quot;'",
   'link': 'https://search.naver.com',
   'description': 'Naver Search Result',
   'lastBuildDate': 'Mon, 20 Mar 2023 20:55:55 +0900',
   'total': '6',
   'start': '1',
   'display': '6',
   'item': [{'title': '<b>겨울왕국</b> 2',
     'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=136873',
     'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1368/136873_P18_100537.jpg',
     'subtitle': 'Frozen 2',
     'pubDate': '2019',
     'director': '크리스 벅|제니퍼 리|',
     'actor': '크리스틴 벨|이디나 멘젤|',
     'userRating': '8.69'},
    {'title': '<b>겨울왕국</b>의 무민',
     'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=163850',
     'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1638/163850_P01_101514.jpg',
     'subtitle': 'Moomins and the Winter Wonderland',
     'pubDate': '2017',
     'director': '이라 카르페란|야쿱 론스키|',
     'actor': '빌 스카스가드|알리시아 비칸데르|스텔란 스카스가드|',
     'userRating': '7.42'},
    {